In [43]:
import IPython.display
import sys
import json
from rdflib import *
import glob
import os
import re
from SPARQLWrapper import SPARQLWrapper, JSON, BASIC

In [44]:
def loadAuthFile():
    with open('auth_add_places.txt') as f:
        auth =  f.readlines()
        auth = [x.replace('\n','') for x in auth]
    return auth

In [45]:
def getSPARQLResults(queryString, auth):
    result = None
    sparql = SPARQLWrapper(auth[0] + "KGRC4SI_20221208_r1")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials(auth[1], auth[2])
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    try :
        json = sparql.query().convert()
        bindings = json['results']['bindings']
        result = bindings
    except  Exception as e:
        print(e)
    return result

In [46]:
def getAskSPARQLResults(queryString, auth):
    result = None
    sparql = SPARQLWrapper(auth[0] + "KGRC4SI_20221208_r1")
    sparql.setHTTPAuth(BASIC)
    sparql.setCredentials(auth[1], auth[2])
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    try :
        json = sparql.query().convert()
        result = json['boolean']
    except  Exception as e:
        print(e)
    return result

In [47]:
def checkClass(instance):
    auth = loadAuthFile()
    query = """
            PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
            ASK { ho:%s ?p ?o }
        """
    result = getAskSPARQLResults(query % class_name, auth)
    return result

In [48]:
def getActivity():
    auth = loadAuthFile()
    query = """
            PREFIX ho: <http://www.owl-ontologies.com/VirtualHome.owl#>
            PREFIX ex: <http://example.org/virtualhome2kg/instance/>
            PREFIX vh2kg: <http://example.org/virtualhome2kg/ontology/>
            SELECT DISTINCT ?s 
            WHERE { 
                ?s vh2kg:virtualHome ?scene . 
            }
        """
    result = getSPARQLResults(query, auth)
    return result

In [49]:
base = Namespace("http://example.org/virtualhome2kg/instance/")
onto = Namespace("http://example.org/virtualhome2kg/ontology/")
ho = Namespace("http://www.owl-ontologies.com/VirtualHome.owl#")
g = Graph()
g.bind("example.py", base)
g.bind("vh2kg", onto)
g.bind("ho", ho)
g.bind("owl", OWL)
results = getActivity()
rules = {
    "clean": "HouseCleaning",
    "television": "Leisure",
    "eat": "EatingDrinking",
    "drink": "EatingDrinking",
    "relax": "Leisure",
    "watch": "Leisure",
    "put": "HouseArrangement"
}
for result in results:
    activity = result["s"]["value"]
    activity_name = activity.replace("http://example.org/virtualhome2kg/instance/", "")
    class_name = re.sub("[0-9]*_scene[0-7]", "", activity_name)
    if checkClass(activity) == False:
        g.add((ho[class_name], RDF.type, OWL.Class))
        parent = [rules[x] for x in rules if x in class_name][0]
        g.add((ho[class_name], RDFS.subClassOf, ho[parent]))

In [50]:
output_path="../dataset/Dataset_12082022_revised/rdf20230228/add_classes.ttl"
g.serialize(destination=output_path, format="ttl")

In [51]:
g.parse("../ontology/vh2kg_schema.ttl", format="ttl")

<Graph identifier=Nf358884aba28424a844660b5ebe607ab (<class 'rdflib.graph.Graph'>)>

In [52]:
output_path="../dataset/Dataset_12082022_revised/rdf20230228/vh2kg_schema_2023-02.ttl"
g.serialize(destination=output_path, format="ttl")